In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('/home/xyz/data/haoyan/bandits/data/Walmart_sales.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data['month'] = data['Date'].dt.month

x = data[['Store', 'month', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']]
y = data['Weekly_Sales']

x_train, x_test, y_train, y_test=train_test_split(x,y,train_size=0.67,random_state=34)

In [3]:
def g(
    max_bin,
    n_estimators,
    max_depth,
    colsample_bytree,
    learning_rate,
    reg_alpha,
    reg_lambda,
):
    config = {
        "max_bin": max_bin,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "colsample_bytree": min(colsample_bytree, 1),
        "learning_rate": learning_rate,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "random_state": 34,
        "tree_method": "hist",
    }

    model = XGBRegressor(**config)
    model.fit(x_train, y_train, verbose=1)
    predictions = model.predict(x_test)
    return -mean_absolute_error(y_test, predictions) / 1e5

In [4]:
n_experiment = 10

# gradient bandit hybrid

In [ ]:
params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBandit(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        init_points=2,
        n_iter=2,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=0.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(1500, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6204   | 0.7623    | 0.2262    | 2.542     | 1.422     |
| 2         | -0.6341   | 0.58      | 0.1928    | 1.417     | 2.085     |
0.5
((256, 5, 500), {'colsample_bytree': 0.7623456721374289, 'learning_rate': 0.2261772009560483, 'reg_alpha': 2.5423948790189437, 'reg_lambda': 1.4217391020307606}) -0.6203943152866055
1
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.7008   | 0.5588    | 0.1881    | 2.556     | 0.6801    |
| 2         | -0.7008   | 0.4781    | 0.1849    | 1.161     | 0.4768    |
0.5
((256, 9, 500), {'colsample_bytree': 0.5587652082636947, 'learning_rate': 0.18805088362141834, 'reg_alpha': 2.5564809351947684, 'reg_lambda': 0.6800677573243523}) -0.7008017838968338
2
|   iter    |  target   | col

| 1         | -0.6624   | 0.7394    | 0.3694    | 1.017     | 1.098     |
| 2         | -0.8382   | 0.4058    | 0.4433    | 1.381     | 0.5558    |
0.5
((512, 8, 500), {'colsample_bytree': 0.7393890775802963, 'learning_rate': 0.3693566356778743, 'reg_alpha': 1.0170087062908166, 'reg_lambda': 1.0983259819733089}) -0.6623545202028897
14
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.7006   | 0.3784    | 0.06916   | 2.274     | 2.479     |
| 2         | -0.6332   | 0.7925    | 0.07813   | 2.951     | 1.886     |
0.5
((64, 5, 2500), {'colsample_bytree': 0.7924738851508153, 'learning_rate': 0.07813156921032059, 'reg_alpha': 2.950761385553745, 'reg_lambda': 1.8861358303555293}) -0.6332452251986229
15
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.689    | 0.93

| 1         | -1.672    | 0.6664    | 0.002468  | 0.4532    | 2.259     |
| 2         | -0.6797   | 0.4434    | 0.0763    | 0.3269    | 1.29      |
0.5
((64, 7, 1000), {'colsample_bytree': 0.44341706987102036, 'learning_rate': 0.07630444953016856, 'reg_alpha': 0.3269058851069021, 'reg_lambda': 1.2896318004258092}) -0.6797194610890419
27
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6785   | 0.9579    | 0.2929    | 2.653     | 0.2532    |
| 2         | -0.6518   | 0.5904    | 0.205     | 2.275     | 0.5628    |
0.5
((512, 8, 2500), {'colsample_bytree': 0.5903623149868511, 'learning_rate': 0.2050278631601805, 'reg_alpha': 2.2752696869474134, 'reg_lambda': 0.5627815038508137}) -0.6518336398511064
28
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 3         | -0.6834   | 0

| 1         | -0.7553   | 0.9956    | 0.4496    | 0.08646   | 0.4005    |
| 2         | -0.8719   | 0.3278    | 0.4938    | 1.673     | 2.459     |
0.5
((256, 8, 1000), {'colsample_bytree': 0.9956186168695862, 'learning_rate': 0.4496185022928139, 'reg_alpha': 0.08645843445020973, 'reg_lambda': 0.40053442154693064}) -0.7552989533839455
40
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.694    | 0.4972    | 0.2007    | 2.082     | 1.477     |
| 2         | -0.8856   | 0.2694    | 0.07148   | 1.887     | 0.02331   |
0.5
((128, 6, 2500), {'colsample_bytree': 0.4971689922471271, 'learning_rate': 0.20066889338128857, 'reg_alpha': 2.081552313060421, 'reg_lambda': 1.476743085739563}) -0.6939572851996528
41
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.8891   | 0

| 1         | -0.6519   | 0.4919    | 0.313     | 2.331     | 2.689     |
| 2         | -0.6382   | 0.7426    | 0.08003   | 0.2289    | 0.1648    |
0.5
((512, 9, 2500), {'colsample_bytree': 0.7425877706784285, 'learning_rate': 0.08002760596473105, 'reg_alpha': 0.22894374012658847, 'reg_lambda': 0.16477316219141336}) -0.6381741308910076
53
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -3.234    | 0.4304    | 0.001329  | 2.712     | 2.839     |
| 2         | -0.6525   | 0.8759    | 0.2861    | 2.127     | 2.508     |
0.5
((64, 6, 1000), {'colsample_bytree': 0.8758920608163716, 'learning_rate': 0.2860738166058937, 'reg_alpha': 2.1266699124218764, 'reg_lambda': 2.5084789639899556}) -0.6524738915048257
54
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 3         | -0.8429   | 

...

| 15        | -0.6685   | 0.9777    | 0.1986    | 2.055     | 0.7762    |
| 16        | -0.6869   | 0.2859    | 0.02116   | 2.28      | 1.829     |
0.5
((128, 5, 5000), {'colsample_bytree': 0.948062152867178, 'learning_rate': 0.11924827996728478, 'reg_alpha': 0.8123246424311977, 'reg_lambda': 0.3813811549538464}) -0.6370264049778941
966
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 19        | -0.6649   | 1.0       | 0.5       | 3.0       | 2.54      |
| 20        | -0.664    | 0.9943    | 0.4925    | 2.221     | 1.948     |
0.5
((256, 4, 500), {'colsample_bytree': 1.0, 'learning_rate': 0.2811287023185303, 'reg_alpha': 3.0, 'reg_lambda': 1.1866904121155375}) -0.6116592834157251
967
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 23        | -0.6593   | 1.0       | 0.001     | 0.265  

| 17        | -0.6374   | 0.6482    | 0.07919   | 1.211     | 1.291     |
| 18        | -0.6991   | 0.6016    | 0.3432    | 1.295     | 0.9908    |
0.5
((128, 5, 2500), {'colsample_bytree': 1.0, 'learning_rate': 0.012946468283431527, 'reg_alpha': 1.7735545314182373, 'reg_lambda': 1.574324321427556}) -0.6017989490554378
979
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 13        | -1.183    | 1.0       | 0.001     | 0.7371    | 1.982     |
| 14        | -0.6444   | 1.0       | 0.5       | 0.0       | 1.863     |
0.5
((256, 6, 2500), {'colsample_bytree': 0.9312802094485111, 'learning_rate': 0.09535983068312921, 'reg_alpha': 1.6757012964608513, 'reg_lambda': 2.5286786889939346}) -0.5984199508562853
980
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 27        | -0.6508   | 1.0       | 0

| 15        | -0.7155   | 0.3072    | 0.1582    | 1.443     | 1.872     |
| 16        | -3.447    | 0.8713    | 0.001     | 1.048     | 1.545     |
0.5
((128, 6, 500), {'colsample_bytree': 0.7459192463409015, 'learning_rate': 0.1954617699668659, 'reg_alpha': 1.9731965764641735, 'reg_lambda': 2.9992725062253767}) -0.6264410789937912
992
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 23        | -0.673    | 1.0       | 0.001     | 3.0       | 1.131     |
| 24        | -0.6514   | 0.9579    | 0.02635   | 1.633     | 0.8655    |
0.5
((64, 9, 5000), {'colsample_bytree': 0.8849826560374474, 'learning_rate': 0.004149929393508352, 'reg_alpha': 1.7057419248849903, 'reg_lambda': 2.882334357231694}) -0.6178598072946093
993
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 13        | -1.725    | 0

| 13        | -0.6382   | 0.726     | 0.2388    | 2.883     | 0.718     |
| 14        | -0.6509   | 0.935     | 0.3188    | 0.03595   | 1.207     |
0.5
((64, 5, 500), {'colsample_bytree': 0.6072561983734805, 'learning_rate': 0.13102270322747053, 'reg_alpha': 0.5672685111701695, 'reg_lambda': 2.3176778529079467}) -0.6248034441222339
1005
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 33        | -3.154    | 1.0       | 0.001     | 1.432     | 3.0       |
| 34        | -0.8663   | 0.2       | 0.5       | 0.6464    | 3.0       |
0.5
((512, 7, 500), {'colsample_bytree': 0.808570581709126, 'learning_rate': 0.13057997448926018, 'reg_alpha': 0.5652087534534973, 'reg_lambda': 1.4882928348818394}) -0.5979834328183852
1006
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 29        | -0.7213   | 

| 31        | -4.575    | 0.2       | 0.001     | 0.9864    | 0.0       |
| 32        | -4.578    | 0.2       | 0.001     | 3.0       | 3.0       |
0.5
((128, 4, 500), {'colsample_bytree': 0.9685861648650833, 'learning_rate': 0.1194104919578715, 'reg_alpha': 1.0050106908990455, 'reg_lambda': 1.0317501933726656}) -0.6196329903675258
1018
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 17        | -0.9108   | 0.239     | 0.4835    | 2.427     | 0.01783   |
| 18        | -0.6194   | 0.5743    | 0.08886   | 2.627     | 1.284     |
0.5
((256, 6, 2500), {'colsample_bytree': 0.9312802094485111, 'learning_rate': 0.09535983068312921, 'reg_alpha': 1.6757012964608513, 'reg_lambda': 2.5286786889939346}) -0.5984199508562853
1019
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 7         | -0.6123   

In [6]:
scores_list = []
actions_list = []
for x in res:
    scores_list.append(x[0])
    actions_list.append(x[1])

NameError: name 'res' is not defined

In [ ]:
for scores, actions in zip(scores_list, actions_list):
    print(actions[np.argmax(scores)], scores[np.argmax(scores)], len(actions))

# gradient bandit random search

In [7]:
params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_estimators": [100, 500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBanditRandomSearch(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        max_single_discrete_param_n_iter=300,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=0.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(600, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
0.01
((256, 3, 5000), ({'colsample_bytree': 0.9088623519724577, 'learning_rate': 0.09252775479126046, 'reg_alpha': 2.661254733388616, 'reg_lambda': 0.16962686687613882}, {'max_bin': 256, 'max_depth': 3, 'n_estimators': 5000})) -0.6564767188301849
1
0.01
((16, 4, 5000), ({'colsample_bytree': 0.47552555662018686, 'learning_rate': 0.036370488973076986, 'reg_alpha': 1.1698756365655822, 'reg_lambda': 1.14154324892839}, {'max_bin': 16, 'max_depth': 4, 'n_estimators': 5000})) -1.1774656444275247
2
0.01
((64, 4, 100), ({'colsample_bytree': 0.7044285708342211, 'learning_rate': 0.0931157957153011, 'reg_alpha': 1.665212254259142, 'reg_lambda': 1.0992101779687005}, {'max_bin': 64, 'max_depth': 4, 'n_estimators': 100})) -1.115862739277307
3


KeyboardInterrupt: 

# random search

In [8]:
import time
import numpy as np
import scipy.stats as stats

from sklearn.model_selection import ParameterSampler


# specify parameters and distributions to sample from
params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "n_estimators": [100, 1000, 2000, 5000],
    "max_depth": [2, 3, 4, 5, 6],
    "colsample_bytree": stats.uniform(0.2, 1),
    'learning_rate': stats.uniform(0.001, 0.5),
    'reg_alpha': stats.uniform(0, 3),
    'reg_lambda': stats.uniform(0, 3),
}

# n_iter = max([len(actions) for actions in actions_list])
n_iter = 600
n_experiment = 10

res_dict_list = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt', 'a') as f:
    for n in range(n_experiment):
        param_list = list(ParameterSampler(params_list_dict, n_iter=n_iter, random_state=n*128))
        res_dict = {}
        for i, param in enumerate(param_list):
            start = time.time()
            res_dict[str(param)] = g(**param)
            end = time.time()
            f.write(f'({n*128}, {i}, {param}, {res_dict[str(param)]}, {end-start})\n')
            if i % 10 == 0:
                print(i, param, res_dict[str(param)])
            
        res_dict_list.append(res_dict)

In [ ]:
for res_dict in res_dict_list:
    inverse = [(value, key) for key, value in res_dict.items()]
    print(max(inverse))

# bayesian opt

In [11]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

def g_hat(max_bin_exp,
        n_estimators,
        max_depth,
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,):
    estimator_list = np.array([100, 1000, 2000, 5000])
    tmp = abs(estimator_list - n_estimators)
    n_estimators = estimator_list[np.argmin(tmp)]
    return g(
        2**int(max_bin_exp),
        n_estimators,
        int(max_depth),
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,
    )

pbounds = {
    'max_bin_exp': (4, 10),
    'n_estimators': (100, 5000),
    "max_depth": (1, 6),
    "colsample_bytree": (0.2, 1),
    'learning_rate': (0.001, 0.5),
    'reg_alpha': (0, 3),
    'reg_lambda': (0, 3),
}

b_res_list = []
for n in range(n_experiment):
    optimizer = BayesianOptimization(
        f=g_hat,
        pbounds=pbounds,
        random_state=n*128,
        allow_duplicate_points=False
    )
    logger = JSONLogger(path=f"/home/xyz/data/haoyan/bandits/logs/bayesian_results_{n*128}.log")
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

    optimizer.maximize(init_points=3, n_iter=597)
    b_res_list.append(optimizer.max)

In [ ]:
[(x['target'], str({'u': int(int(x['params']['u'])/n_interval_u) * n_interval_u + 1, 'v': int(int(x['params']['v'])/n_interval_v) * n_interval_v + 1, 'w': x['params']['w'], 'x': int(int(x['params']['x'])/n_interval_x) * n_interval_x + 1, 'y': x['params']['y'], 'z': x['params']['z']})) for x in b_res_list]

[(0.2768580338893678,
  "{'u': 4, 'v': 7, 'w': 5.116789806249014, 'x': 13, 'y': 10.080905428791336, 'z': 0.9911011711627078}"),
 (0.1402183484847389,
  "{'u': 4, 'v': 7, 'w': 5.011204786829208, 'x': 1, 'y': 10.000305831311975, 'z': 0.9868788787576984}"),
 (0.27722306853469403,
  "{'u': 4, 'v': 7, 'w': 4.902417985793109, 'x': 13, 'y': 9.945820186243077, 'z': 0.924097238551662}"),
 (0.2768119314094586,
  "{'u': 4, 'v': 7, 'w': 4.885291040699603, 'x': 13, 'y': 10.081867600308184, 'z': 1.0257453084676125}"),
 (0.2768976627971634,
  "{'u': 4, 'v': 7, 'w': 5.09077481582764, 'x': 13, 'y': 9.937800034531513, 'z': 0.9109290708881388}"),
 (0.2799674220581756,
  "{'u': 4, 'v': 7, 'w': 4.948148565613696, 'x': 13, 'y': 10.012700410959875, 'z': 0.9925553408392699}"),
 (0.2753907393071401,
  "{'u': 4, 'v': 7, 'w': 5.075218248621907, 'x': 13, 'y': 9.860434685868848, 'z': 0.9419097111446684}"),
 (0.13862511632972735,
  "{'u': 4, 'v': 7, 'w': 5.048247231101259, 'x': 1, 'y': 10.066125798110978, 'z': 0.89

# rl

In [ ]:
n_trials = 600

params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "n_estimators": [100, 1000, 2000, 5000],
    "max_depth": [2, 3, 4, 5, 6],
    "colsample_bytree": np.linspace(0.2, 1, 10),
    'learning_rate': np.linspace(0.001, 0.5, 30),
    'reg_alpha': np.linspace(0, 3, 30),
    'reg_lambda': np.linspace(0, 3, 30),
}

scores_list_ori = []
actions_list_ori = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_results.txt', 'a') as f:

    for n in range(n_experiment):
        bandit = bd.GaussianBanditOri(
            g,
            params_list_dict=params_list_dict,
        )

        policy = bd.SoftmaxPolicy()
        agent = bd.GradientAgent(bandit, policy, alpha=0.1, baseline=True, increase_rate=0)
        env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
        scores, actions = env.run(n_trials, file=f)

        idx = np.argmax(scores)

        scores_list_ori.append(scores)
        actions_list_ori.append(actions)

0
0.1
((1024, 2000, 6, 0.9111111111111112, 0.3279310344827586, 2.2758620689655173, 2.0689655172413794), None) -0.6260685358179585
1
0.1
((1024, 100, 5, 0.37777777777777777, 0.3107241379310345, 0.20689655172413793, 1.5517241379310345), None) -0.8328349966322386
2
0.1
((64, 5000, 6, 0.2888888888888889, 0.15586206896551724, 0.41379310344827586, 2.0689655172413794), None) -0.8418240917834782
3
0.1
((128, 2000, 3, 0.2, 0.20748275862068966, 2.793103448275862, 2.2758620689655173), None) -0.8793157143802965
4
0.1
((256, 2000, 5, 0.37777777777777777, 0.19027586206896552, 2.4827586206896552, 1.9655172413793103), None) -0.7167884216563309
5
0.1
((256, 1000, 4, 0.7333333333333334, 0.2246896551724138, 2.793103448275862, 3.0), None) -0.6198023924045138
6
0.1
((32, 1000, 3, 0.8222222222222222, 0.4655862068965517, 0.5172413793103449, 0.5172413793103449), None) -0.7707729103842986
7
0.1
((128, 100, 2, 0.7333333333333334, 0.001, 1.7586206896551724, 1.4482758620689655), None) -4.656989500353108
8
0.1
((6

0.1
((16, 2000, 4, 0.2, 0.3623448275862069, 2.689655172413793, 0.3103448275862069), None) -3.1434687779822856
67
0.1
((16, 5000, 4, 0.7333333333333334, 0.17306896551724138, 0.8275862068965517, 2.1724137931034484), None) -1.0883461385813915
68
0.1
((128, 5000, 6, 0.2888888888888889, 0.4483793103448276, 1.8620689655172413, 2.689655172413793), None) -0.8829803433260414
69
0.1
((1024, 1000, 6, 0.5555555555555556, 0.24189655172413793, 2.4827586206896552, 1.4482758620689655), None) -0.6459754424243762
70
0.1
((256, 2000, 4, 0.9111111111111112, 0.06982758620689655, 2.1724137931034484, 0.10344827586206896), None) -0.6265455392383181
71
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 0.5172413793103449, 0.0), None) -1.3235876037238112
72
0.1
((512, 1000, 5, 0.9111111111111112, 0.3967586206896552, 2.1724137931034484, 0.20689655172413793), None) -0.6601016519479754
73
0.1
((1024, 5000, 5, 0.8222222222222222, 0.08703448275862069, 0.20689655172413793, 2.586206896551724), None) -0.5977373

0.1
((1024, 1000, 5, 0.7333333333333334, 0.17306896551724138, 1.1379310344827587, 0.41379310344827586), None) -0.6130731549817561
134
0.1
((32, 100, 2, 0.2888888888888889, 0.2763103448275862, 0.3103448275862069, 2.2758620689655173), None) -2.2348664159545666
135
0.1
((64, 2000, 5, 0.4666666666666667, 0.19027586206896552, 3.0, 0.5172413793103449), None) -0.7251769841447446
136
0.1
((16, 1000, 6, 0.9111111111111112, 0.17306896551724138, 1.2413793103448276, 2.3793103448275863), None) -1.0351576604946444
137
0.1
((64, 100, 4, 0.7333333333333334, 0.3967586206896552, 0.6206896551724138, 2.793103448275862), None) -0.6688710296418904
138
0.1
((128, 100, 3, 1.0, 0.19027586206896552, 0.7241379310344828, 0.6206896551724138), None) -0.8355957175729757
139
0.1
((64, 100, 4, 0.2, 0.24189655172413793, 2.896551724137931, 1.4482758620689655), None) -1.6234752791696092
140
0.1
((256, 100, 6, 0.2888888888888889, 0.1386551724137931, 1.8620689655172413, 0.9310344827586207), None) -0.9838842335657955
141
0.

0.1
((64, 2000, 4, 0.2, 0.01820689655172414, 2.4827586206896552, 1.0344827586206897), None) -1.5049309910516715
200
0.1
((512, 2000, 4, 0.37777777777777777, 0.10424137931034483, 2.0689655172413794, 1.9655172413793103), None) -0.6658038511202422
201
0.1
((256, 1000, 2, 0.5555555555555556, 0.19027586206896552, 0.7241379310344828, 1.4482758620689655), None) -0.7442135368570504
202
0.1
((16, 5000, 6, 0.2888888888888889, 0.3107241379310345, 1.7586206896551724, 2.689655172413793), None) -1.4252912974035574
203
0.1
((256, 100, 2, 0.2, 0.3279310344827586, 2.896551724137931, 2.689655172413793), None) -2.5765958818032013
204
0.1
((16, 5000, 4, 0.8222222222222222, 0.43117241379310345, 0.41379310344827586, 2.0689655172413794), None) -1.1573290225671455
205
0.1
((16, 1000, 3, 0.9111111111111112, 0.12144827586206897, 1.1379310344827587, 1.5517241379310345), None) -1.2582422880303965
206
0.1
((128, 2000, 2, 0.6444444444444445, 0.3623448275862069, 2.4827586206896552, 0.8275862068965517), None) -0.7015

0.1
((32, 2000, 4, 1.0, 0.17306896551724138, 0.3103448275862069, 1.2413793103448276), None) -0.7497864386277366
266
0.1
((1024, 5000, 6, 0.2, 0.24189655172413793, 2.2758620689655173, 2.3793103448275863), None) -0.9694537991393009
267
0.1
((64, 2000, 2, 0.9111111111111112, 0.43117241379310345, 1.8620689655172413, 1.8620689655172413), None) -0.7056774155985168
268
0.1
((1024, 1000, 5, 0.2, 0.10424137931034483, 0.3103448275862069, 1.7586206896551724), None) -0.8573869944812265
269
0.1
((512, 1000, 3, 0.5555555555555556, 0.37955172413793103, 0.7241379310344828, 1.3448275862068966), None) -0.6789993165563207
270
0.1
((64, 2000, 3, 0.8222222222222222, 0.29351724137931035, 2.793103448275862, 0.6206896551724138), None) -0.6839185483779131
271
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 2.4827586206896552, 1.4482758620689655), None) -1.3204220489222411
272
0.1
((64, 5000, 3, 0.8222222222222222, 0.4655862068965517, 2.896551724137931, 1.3448275862068966), None) -0.7640960120542019


...

0.1
((512, 1000, 3, 0.5555555555555556, 0.1386551724137931, 0.6206896551724138, 3.0), None) -0.6738574899603488
251
0.1
((32, 100, 5, 0.2888888888888889, 0.001, 0.41379310344827586, 2.793103448275862), None) -4.6605426274835215
252
0.1
((512, 100, 4, 0.2888888888888889, 0.3623448275862069, 0.6206896551724138, 1.0344827586206897), None) -0.8767266060469633
253
0.1
((32, 5000, 3, 1.0, 0.08703448275862069, 2.793103448275862, 0.20689655172413793), None) -0.7429487315674288
254
0.1
((16, 1000, 5, 0.8222222222222222, 0.19027586206896552, 1.2413793103448276, 2.689655172413793), None) -1.0517050458737935
255
0.1
((256, 100, 6, 0.2888888888888889, 0.12144827586206897, 1.6551724137931034, 2.896551724137931), None) -1.0276292603048494
256
0.1
((64, 100, 4, 1.0, 0.3623448275862069, 2.586206896551724, 1.9655172413793103), None) -0.6317370445386065
257
0.1
((1024, 100, 3, 0.8222222222222222, 0.25910344827586207, 2.0689655172413794, 2.1724137931034484), None) -0.8073495081479518
258
0.1
((32, 1000, 4

0.1
((256, 100, 3, 0.8222222222222222, 0.25910344827586207, 1.5517241379310345, 0.8275862068965517), None) -0.7963603194503296
318
0.1
((1024, 1000, 4, 0.37777777777777777, 0.24189655172413793, 1.2413793103448276, 2.689655172413793), None) -0.6846141769034692
319
0.1
((32, 100, 2, 0.37777777777777777, 0.5, 3.0, 2.2758620689655173), None) -1.6838373549111347
320
0.1
((128, 5000, 5, 0.5555555555555556, 0.20748275862068966, 0.8275862068965517, 1.8620689655172413), None) -0.6986671263933028
321
0.1
((256, 5000, 3, 1.0, 0.4483793103448276, 1.0344827586206897, 2.4827586206896552), None) -0.7200505861816177
322
0.1
((512, 1000, 5, 0.37777777777777777, 0.3107241379310345, 2.0689655172413794, 3.0), None) -0.699354004497475
323
0.1
((64, 2000, 2, 1.0, 0.4483793103448276, 2.2758620689655173, 0.5172413793103449), None) -0.7118250286885005
324
0.1
((32, 1000, 6, 0.37777777777777777, 0.2246896551724138, 1.7586206896551724, 1.9655172413793103), None) -0.8521338136556911
325
0.1
((128, 100, 3, 0.55555

0.1
((256, 5000, 2, 0.37777777777777777, 0.10424137931034483, 2.586206896551724, 2.1724137931034484), None) -0.7412676933380414
384
0.1
((512, 1000, 3, 1.0, 0.5, 0.8275862068965517, 2.2758620689655173), None) -0.6658537372116751
385
0.1
((256, 5000, 4, 0.5555555555555556, 0.29351724137931035, 0.0, 2.3793103448275863), None) -0.7114154139481079
386
0.1
((64, 100, 6, 0.37777777777777777, 0.24189655172413793, 0.0, 2.3793103448275863), None) -0.7835094865193915
387
0.1
((128, 100, 4, 0.6444444444444445, 0.37955172413793103, 0.3103448275862069, 0.7241379310344828), None) -0.6903573071195269
388
0.1
((256, 100, 2, 0.8222222222222222, 0.3279310344827586, 0.0, 2.0689655172413794), None) -1.0233757000706214
389
0.1
((64, 1000, 6, 0.9111111111111112, 0.19027586206896552, 2.793103448275862, 1.6551724137931034), None) -0.6376681099774895
390
0.1
((128, 100, 3, 1.0, 0.3107241379310345, 1.2413793103448276, 1.2413793103448276), None) -0.743894241682483
391
0.1
((256, 2000, 5, 0.37777777777777777, 0.5

0.1
((16, 2000, 4, 0.9111111111111112, 0.20748275862068966, 1.8620689655172413, 0.7241379310344828), None) -1.0870106469624035
451
0.1
((1024, 2000, 5, 0.8222222222222222, 0.15586206896551724, 2.4827586206896552, 1.0344827586206897), None) -0.6011991885637359
452
0.1
((512, 2000, 3, 0.6444444444444445, 0.052620689655172415, 2.3793103448275863, 2.1724137931034484), None) -0.6509203497108933
453
0.1
((16, 1000, 3, 0.5555555555555556, 0.2246896551724138, 1.6551724137931034, 0.5172413793103449), None) -1.2736633029013653
454
0.1
((16, 2000, 2, 0.8222222222222222, 0.08703448275862069, 0.9310344827586207, 0.20689655172413793), None) -1.7516956391550065
455
0.1
((32, 2000, 4, 0.37777777777777777, 0.48279310344827586, 1.4482758620689655, 2.0689655172413794), None) -0.8752010279756649
456
0.1
((512, 1000, 6, 0.5555555555555556, 0.29351724137931035, 0.3103448275862069, 2.586206896551724), None) -0.6575992973995114
457
0.1
((64, 1000, 3, 0.5555555555555556, 0.001, 1.7586206896551724, 0.0), None) 

0.1
((32, 2000, 2, 0.6444444444444445, 0.29351724137931035, 0.0, 2.4827586206896552), None) -0.8103826461143478
517
0.1
((128, 5000, 3, 0.6444444444444445, 0.43117241379310345, 1.2413793103448276, 0.10344827586206896), None) -0.7691932451575786
518
0.1
((128, 1000, 2, 0.6444444444444445, 0.24189655172413793, 0.9310344827586207, 3.0), None) -0.7239669985331333
519
0.1
((512, 5000, 2, 0.5555555555555556, 0.08703448275862069, 1.9655172413793103, 0.0), None) -0.7091701003759123
520
0.1
((1024, 2000, 6, 0.6444444444444445, 0.03541379310344828, 0.10344827586206896, 1.1379310344827587), None) -0.6048283081523658
521
0.1
((512, 1000, 2, 0.4666666666666667, 0.29351724137931035, 2.896551724137931, 1.1379310344827587), None) -0.720744961880223
522
0.1
((16, 2000, 6, 0.2888888888888889, 0.4139655172413793, 0.0, 0.9310344827586207), None) -1.3879239488935968
523
0.1
((16, 2000, 3, 0.37777777777777777, 0.19027586206896552, 1.8620689655172413, 0.6206896551724138), None) -1.416350976850779
524
0.1
((5

In [ ]:
for scores, actions in zip(scores_list_ori, actions_list_ori):
    print(actions[np.argmax(scores)][0], scores[np.argmax(scores)])

(335, 4.348633064005995) 0.03436411662463359
(375, 8.056192227792936) 0.03387046171330028
(267, 12.158172579216787) 0.03839516208255515
(412, 7.819539515210791) 0.03255006780027559
(449, 11.44821444147035) 0.033082805223584796
(365, 5.610780864444103) 0.032638915566141534
(426, 4.664170014115522) 0.0322694725026607
(480, 3.9542118763690866) 0.03496507206821923
(391, 8.68726612801199) 0.03557410140924732
(124, 9.239455790703662) 0.04142491925257696


our hybrid optimization method takes advantage of reinforcement learning to handle the discrete parameters, which successfully reduces the dimension of the parameter space for Bayesian Optimization. The technique improves the accuracy
of the solutions, compared with Bayesian Optimization and Random Search. The overhead of the one step Bayesian Optimization
is overcomed by highly reducing the iteration steps of Bayesian Optimization in the hybrid method because of the 
dimension reduction. The overall iteration steps is also compressed compared with Random Search to reach the global
minimizer.